# 6. OCR文本错漏频发？结合LLM纠错，让图像文本也能精准使用

### 1.1 错误类型分析

OCR识别错误通常可以分为以下几类：

- **字符识别错误**
- **文字遗漏**
- **多字重复**
- **格式混乱**
- **特殊符号识别错误**

这些错误往往源于以下几个方面：

- **图像质量不佳**
- **字体样式复杂**
- **背景干扰**
- **OCR算法限制**

### 1.2 传统解决方案的局限性

传统的OCR优化方法主要包括：

- **图像预处理**
- **后处理规则**
- **模型微调**

## 结合 LLM 进行文本纠错的新思路
- 充分发挥 LLM 的语言建模能力
- OCR + LLM 的协同流程

我们可以将整个OCR处理流程分为两个阶段：

1. OCR识别阶段：使用 PaddleOCR 5 对输入图像进行识别，得到初步的文本结果。
2. LLM纠错阶段：将OCR输出的文本送入大语言模型，由其进行语义级别的纠错和优化。

具体流程如下：
[图像] → [PaddleOCR 5] → [初步OCR文本] → [LLM纠错] → [最终文本]

# 实战操作

[快速安装飞桨平台工具](https://www.paddlepaddle.org.cn/install/quick?docurl=undefined)



In [1]:
# 安装 PaddleOCR
! pip install paddleocr

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple


![OCR 示例图片](https://paddle-model-ecology.bj.bcebos.com/paddlex/imgs/demo_image/general_ocr_002.png  "通用 OCR 示例图像")

In [ ]:
# 运行 PP-OCRv5 推理
! paddleocr ocr -i https://paddle-model-ecology.bj.bcebos.com/paddlex/imgs/demo_image/general_ocr_002.png --use_doc_orientation_classify False --use_doc_unwarping False --use_textline_orientation False 

# 运行 PP-ChatOCRv4 推理前，需要先获得千帆API Key
# paddleocr pp_chatocrv4_doc -i https://paddle-model-ecology.bj.bcebos.com/paddlex/imgs/demo_image/vehicle_certificate-1.png -k 驾驶室准乘人数 --qianfan_api_key your_api_key --use_doc_orientation_classify False --use_doc_unwarping False 

# 查看 "paddleocr ocr" 详细参数
# paddleocr ocr --help

In [7]:
# PP-OCRv5 示例
from paddleocr import PaddleOCR
# 初始化 PaddleOCR 实例
ocr = PaddleOCR(
    use_doc_orientation_classify=False,
    use_doc_unwarping=False,
    use_textline_orientation=False)
# 对示例图像执行 OCR 推理 
result = ocr.predict(
    input="https://paddle-model-ecology.bj.bcebos.com/paddlex/imgs/demo_image/general_ocr_002.png")
# 可视化结果并保存 json 结果
for res in result:
    res.print()
    res.save_to_img("output")
    res.save_to_json("output")

MKL-DNN is not available. Using `paddle` instead.
Creating model: ('PP-OCRv5_server_det', None)
Using official model (PP-OCRv5_server_det), the model files will be automatically downloaded and saved in /Users/wilson/.paddlex/official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 4360.74it/s]
Creating model: ('PP-OCRv5_server_rec', None)
Using official model (PP-OCRv5_server_rec), the model files will be automatically downloaded and saved in /Users/wilson/.paddlex/official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 4747.37it/s]
Connecting to https://paddle-model-ecology.bj.bcebos.com/paddlex/imgs/demo_image/general_ocr_002.png ...
[==================================================] 100.00%
{'res': {'input_path': '/Users/wilson/.paddlex/predict_input/general_ocr_002.png', 'page_index': None, 'model_settings': {'use_doc_preprocessor': True, 'use_textline_orientation': False}, 'doc_preprocessor_res': {'input_path': None, 'page_index': None, 'model_settings

![PP-OCRv5](https://paddle-model-ecology.bj.bcebos.com/paddlex/PaddleX3.0/AIStudio/application_help/demo_images/algorithm_ppocrv5.png)

![PP-ChatOCRV4](https://paddle-model-ecology.bj.bcebos.com/paddlex/PaddleX3.0/AIStudio/application_help/demo_images/algorithm_ppchatocrv4.png)

![ocr_result](https://paddle-model-ecology.bj.bcebos.com/paddlex/imgs/demo_image/vehicle_certificate-1.png)

In [13]:
# PP-ChatOCRv4 示例
from paddleocr import PPChatOCRv4Doc
import  os
BCE_API_KEY = os.environ.get("BCE_API_KEY")

chat_bot_config = {
    "module_name": "chat_bot",
    "model_name": "ernie-4.0-8k",
    "base_url": "https://qianfan.baidubce.com/v2",
    "api_type": "openai",
    "api_key": BCE_API_KEY,  # your api_key
}

retriever_config = {
    "module_name": "retriever",
    "model_name": "embedding-v1",
    "base_url": "https://qianfan.baidubce.com/v2",
    "api_type": "qianfan",
    "api_key": BCE_API_KEY,  # your api_key
}

pipeline = PPChatOCRv4Doc(
    use_doc_orientation_classify=False,
    use_doc_unwarping=False
)

visual_predict_res = pipeline.visual_predict(
    input="https://paddle-model-ecology.bj.bcebos.com/paddlex/imgs/demo_image/vehicle_certificate-1.png",
    use_common_ocr=True,
    use_seal_recognition=True,
    use_table_recognition=True,
)

mllm_predict_info = None
use_mllm = False
# 如果使用多模态大模型，需要启动本地 mllm 服务，可以参考文档：https://github.com/PaddlePaddle/PaddleX/blob/release/3.0/docs/pipeline_usage/tutorials/vlm_pipelines/doc_understanding.md 进行部署，并更新 mllm_chat_bot_config 配置。
if use_mllm:
    mllm_chat_bot_config = {
        "module_name": "chat_bot",
        "model_name": "PP-DocBee",
        "base_url": "http://127.0.0.1:8080/",  # your local mllm service url
        "api_type": "openai",
        "api_key": "api_key",  # your api_key
    }

    mllm_predict_res = pipeline.mllm_pred(
        input="https://paddle-model-ecology.bj.bcebos.com/paddlex/imgs/demo_image/vehicle_certificate-1.png",
        key_list=["驾驶室准乘人数"],
        mllm_chat_bot_config=mllm_chat_bot_config,
    )
    mllm_predict_info = mllm_predict_res["mllm_res"]

visual_info_list = []
for res in visual_predict_res:
    visual_info_list.append(res["visual_info"])
    layout_parsing_result = res["layout_parsing_result"]

vector_info = pipeline.build_vector(
    visual_info_list, flag_save_bytes_vector=True, retriever_config=retriever_config
)
chat_result = pipeline.chat(
    key_list=["驾驶室准乘人数"],
    visual_info=visual_info_list,
    vector_info=vector_info,
    mllm_predict_info=mllm_predict_info,
    chat_bot_config=chat_bot_config,
    retriever_config=retriever_config,
)

print(chat_result)

MKL-DNN is not available. Using `paddle` instead.
Creating model: ('RT-DETR-H_layout_3cls', None)
Using official model (RT-DETR-H_layout_3cls), the model files will be automatically downloaded and saved in /Users/wilson/.paddlex/official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 3018.21it/s]
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Using official model (PP-LCNet_x1_0_textline_ori), the model files will be automatically downloaded and saved in /Users/wilson/.paddlex/official_models.
Fetching 8 files: 100%|██████████| 8/8 [00:00<00:00, 3872.41it/s]
Creating model: ('PP-OCRv4_server_det', None)
Using official model (PP-OCRv4_server_det), the model files will be automatically downloaded and saved in /Users/wilson/.paddlex/official_models.
Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 3181.12it/s]
Creating model: ('PP-OCRv4_server_rec_doc', None)
Using official model (PP-OCRv4_server_rec_doc), the model files will be automatically downloaded and saved i

{'chat_res': {'驾驶室准乘人数': '2'}}
